## Bias Mitigation using Fairlearn - Heart Failure Prediction Dataset (Source: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction/data)

In [1]:
#load preprocessed data 
import pandas as pd
train_df = pd.read_csv("./data_subsets/train_25M_75F.csv")

X_test = pd.read_csv("./data_splits/X_test.csv")
y_test = pd.read_csv("./data_splits/y_test.csv")

#check out the data
train_df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,33,0,3,100.0,246.0,0,0,150.0,1,1.0,1,1
1,48,0,1,120.0,284.0,0,0,120.0,0,0.0,2,0
2,49,0,3,130.0,269.0,0,0,163.0,0,0.0,2,0
3,62,0,3,140.0,268.0,0,2,160.0,0,3.6,0,1
4,38,0,3,105.0,236.0,1,0,166.0,0,2.8,2,1


In [2]:
# Ensure y_test is a Series (not a DataFrame with 1 column)
y_test = y_test.squeeze("columns")

# Define target and sensitive column names
TARGET = "HeartDisease"
SENSITIVE = "Sex"

# Split train into X/y
X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

# Extract sensitive features separately
A_train = X_train[SENSITIVE].astype(int)
A_test  = X_test[SENSITIVE].astype(int)


In [3]:
TARGET = "HeartDisease"
SENSITIVE = "Sex"   # 1 = Male, 0 = Female

categorical_cols = ['Sex','ChestPainType','FastingBS','RestingECG','ExerciseAngina','ST_Slope']
continuous_cols  = ['Age','RestingBP','Cholesterol','MaxHR','Oldpeak']

In [4]:
# Split train into X / y and keep sensitive feature for fairness evaluation
X_train = train_df.drop(columns=[TARGET])
y_train = train_df[TARGET]

In [5]:
# scale numeric features only, fit on train, transform test
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_num_scaled = pd.DataFrame(
    scaler.fit_transform(X_train[continuous_cols]),
    columns=continuous_cols, index=X_train.index
)
X_test_num_scaled = pd.DataFrame(
    scaler.transform(X_test[continuous_cols]),
    columns=continuous_cols, index=X_test.index
)

In [6]:
#one-hot encode categoricals; numeric are kept as is 
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown="ignore", drop="if_binary", sparse_output=False)
ohe.fit(X_train[categorical_cols])

X_train_cat = pd.DataFrame(
    ohe.transform(X_train[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_train.index
)
X_test_cat = pd.DataFrame(
    ohe.transform(X_test[categorical_cols]),
    columns=ohe.get_feature_names_out(categorical_cols),
    index=X_test.index
)

In [7]:
# Assemble final matrices
X_train_ready = pd.concat([X_train_cat, X_train_num_scaled], axis=1)
X_test_ready  = pd.concat([X_test_cat,  X_test_num_scaled],  axis=1)

print("Final feature shapes:", X_train_ready.shape, X_test_ready.shape)

Final feature shapes: (600, 18) (184, 18)


### Traditional ML Models - Baseline: K-Nearest Neighbors (KNN) & Decision Tree (DT)

In [8]:
#import required libraries
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)

#define a function 
def evaluate_model(y_true, y_pred, model_name):
    print(f"=== {model_name} Evaluation ===")
    print("Accuracy :", accuracy_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred, average='binary'))
    print("Recall   :", recall_score(y_true, y_pred, average='binary'))
    print("F1 Score :", f1_score(y_true, y_pred, average='binary'))
    print("\nClassification Report:\n", classification_report(y_true, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
    print("\n" + "="*40 + "\n")

### PCA-KNN

In [9]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import numpy as np

#1) PCA + KNN pipeline (on one-hot encoded + scaled features)
pca_knn = Pipeline([
    ('pca', PCA(n_components=0.95, random_state=42)),  # keep 95% variance
    ('knn', KNeighborsClassifier(
        n_neighbors=15, metric='manhattan', weights='distance'
    ))
])

pca_knn.fit(X_train_ready, y_train)

# Inspect PCA details
n_comp = pca_knn.named_steps['pca'].n_components_
expl_var = pca_knn.named_steps['pca'].explained_variance_ratio_.sum()

print("=== Baseline (tuned PCA+KNN, no mitigation) ===")
# 2) Evaluate 
y_pred_pca_knn = pca_knn.predict(X_test_ready)
probs_pca_knn = pca_knn.predict_proba(X_test_ready)[:, 1]
  
evaluate_model(y_test, y_pred_pca_knn, "KNN (best params)")

=== Baseline (tuned PCA+KNN, no mitigation) ===
=== KNN (best params) Evaluation ===
Accuracy : 0.8315217391304348
Precision: 0.9080459770114943
Recall   : 0.7745098039215687
F1 Score : 0.8359788359788359

Classification Report:
               precision    recall  f1-score   support

           0       0.76      0.90      0.83        82
           1       0.91      0.77      0.84       102

    accuracy                           0.83       184
   macro avg       0.84      0.84      0.83       184
weighted avg       0.84      0.83      0.83       184

Confusion Matrix:
 [[74  8]
 [23 79]]




### Post-Processing -  KNN

In [10]:
# Demographic Parity post-processing for your tuned PCA+KNN

from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.metrics import (
    MetricFrame, true_positive_rate, false_positive_rate, selection_rate,
    demographic_parity_difference, equalized_odds_difference
)
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd

# Helper function
def eval_fairness(y_true, y_pred, A):
    mf = MetricFrame(
        metrics={
            "TPR": true_positive_rate,
            "FPR": false_positive_rate,
            "Recall": recall_score, 
            "SelectionRate": selection_rate,
            "Accuracy": accuracy_score,
        },
        y_true=y_true, y_pred=y_pred, sensitive_features=A
    )
    return {
        "by_group": mf.by_group,
        "acc": accuracy_score(y_true, y_pred),
        "recall": recall_score(y_true, y_pred),
        "dp": demographic_parity_difference(y_true, y_pred, sensitive_features=A),
        "eo": equalized_odds_difference(y_true, y_pred, sensitive_features=A),
    }

# 1) Baseline metrics (no mitigation) 
pca_knn.fit(X_train_ready, y_train)
y_base = pca_knn.predict(X_test_ready)
m_base = eval_fairness(y_test, y_base, A_test)

print("=== Baseline (tuned PCA+KNN) ===")
print(m_base["by_group"])
print(f"Accuracy: {m_base['acc']:.4f} | DP diff: {m_base['dp']:.4f} | EO diff: {m_base['eo']:.4f}")

# 2) Post-processing with DEMOGRAPHIC PARITY
post_dp = ThresholdOptimizer(
    estimator=pca_knn,
    constraints="demographic_parity",
    predict_method="predict_proba",   # KNN supports this
    grid_size=200,
    flip=True
)
post_dp.fit(X_train_ready, y_train, sensitive_features=A_train)

y_dp = post_dp.predict(X_test_ready, sensitive_features=A_test, random_state=42)
m_dp = eval_fairness(y_test, y_dp, A_test)

print("\n=== Post-processing (Demographic Parity) ===")
print(m_dp["by_group"])
print(f"Accuracy: {m_dp['acc']:.4f} | DP diff: {m_dp['dp']:.4f} | EO diff: {m_dp['eo']:.4f}")

# 3) Post-processing with EQUALIZED ODDS
post_eod = ThresholdOptimizer(
    estimator=pca_knn,
    constraints="equalized_odds",
    predict_method="predict_proba",   
    grid_size=200,
    flip=True,                               
)
post_eod.fit(X_train_ready, y_train, sensitive_features=A_train)

y_eod = post_eod.predict(X_test_ready, sensitive_features=A_test, random_state=42)
m_eod = eval_fairness(y_test, y_eod, A_test)

print("\n=== Post-processing (Equalized Odds) ===")
print(m_eod["by_group"])
print(f"Accuracy: {m_eod['acc']:.4f} | DP diff: {m_eod['dp']:.4f} | EO diff: {m_eod['eo']:.4f}")

=== Baseline (tuned PCA+KNN) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.666667  0.03125  0.666667       0.131579  0.921053
1    0.781250  0.14000  0.781250       0.561644  0.808219
Accuracy: 0.8315 | DP diff: 0.4301 | EO diff: 0.1146

=== Post-processing (Demographic Parity) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.666667  0.03125  0.666667       0.131579  0.921053
1    0.781250  0.14000  0.781250       0.561644  0.808219
Accuracy: 0.8315 | DP diff: 0.4301 | EO diff: 0.1146

=== Post-processing (Equalized Odds) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.666667  0.03125  0.666667       0.131579  0.921053
1    0.781250  0.14000  0.781250       0.561644  0.808219
Accuracy: 0.8315 | DP diff: 0.4301 | EO diff: 0.1146


### Bias Mitigation Results: PCA+KNN – Post-Processing

#### Metrics Overview

| Model                    | Accuracy | DP diff | EO diff | Notes                                                             |
|--------------------------|:--------:|:-------:|:-------:|-------------------------------------------------------------------|
| PCA+KNN Baseline (tuned) | 0.8315   | 0.4301  | 0.1146  | Large DP gap; moderate EO gap                                     |
| PCA+KNN + PP (DP)        | 0.8315   | 0.4301  | 0.1146  | **Identical to baseline** → post-processing not applied/effective |
| PCA+KNN + PP (EO)        | 0.8315   | 0.4301  | 0.1146  | **Identical to baseline** → post-processing not applied/effective |

---

#### Interpretation
- Baseline shows **strong outcome disparity** (DP ≈ 0.43) and **non-trivial error-rate gap** (EO ≈ 0.115).
- Both post-processing runs show **no change**, suggesting scores weren’t used or mitigation wasn’t applied at inference.


---

**CorrelationRemover** will be implemented to improve fairness after DP/EOD post-processing failed to change any predictions (0% flips), leaving metrics unchanged. By removing linear correlation between features and the sensitive attribute, we reduce leakage and make group score distributions more comparable, giving PCA+KNN and also any subsequent post-processing room to adjust selection rates and error rates—all while staying.

In [11]:
from fairlearn.preprocessing import CorrelationRemover
from sklearn.metrics import recall_score  

Xtr_df = X_train_ready.copy()
Xte_df = X_test_ready.copy()
Xtr_df["__A__"] = A_train.values
Xte_df["__A__"] = A_test.values

cr = CorrelationRemover(sensitive_feature_ids=["__A__"])

Xtr_fair_arr = cr.fit_transform(Xtr_df)   # shape: (n_samples, n_features - 1)
Xte_fair_arr = cr.transform(Xte_df)

# Rebuild DataFrames with columns that exclude the sensitive column
cols_out = [c for c in Xtr_df.columns if c != "__A__"]
Xtr_fair = pd.DataFrame(Xtr_fair_arr, index=Xtr_df.index, columns=cols_out)
Xte_fair = pd.DataFrame(Xte_fair_arr, index=Xte_df.index, columns=cols_out)

# Refit your PCA+KNN
pca_knn.fit(Xtr_fair, y_train)
y_cr = pca_knn.predict(Xte_fair)
m_cr = eval_fairness(y_test, y_cr, A_test)

print("\n=== Preprocessing: CorrelationRemover + PCA+KNN ===")
print(m_cr["by_group"])
print(f"Accuracy: {m_cr['acc']:.4f} | DP diff: {m_cr['dp']:.4f} | EO diff: {m_cr['eo']:.4f}")


=== Preprocessing: CorrelationRemover + PCA+KNN ===
          TPR    FPR    Recall  SelectionRate  Accuracy
Sex                                                    
0    0.500000  0.125  0.500000       0.184211  0.815789
1    0.802083  0.200  0.802083       0.595890  0.801370
Accuracy: 0.8043 | DP diff: 0.4117 | EO diff: 0.3021


**Interpretation**:
- Accuracy **0.8043** (↓ vs baseline **0.8315**, −2.72 pts).
- **DP diff = 0.4117**: selection rates **Female 0.184** vs **Male 0.596** (~**3.23×** higher for males) → under-selection of females.
- **EO diff = 0.3021** (worse than baseline **0.1146**):
  - **TPR:** Female **0.500** vs Male **0.802** (gap **0.302**) → more missed positives among females.
  - **FPR:** Female **0.125** vs Male **0.200** (gap **0.075**) → more false alarms among males.
- Group accuracy: **Female 0.816** vs **Male 0.801** (gap ~**0.014**, smaller than baseline).

CorrelationRemover slightly improves **DP** and narrows **FPR/accuracy gaps**, but **worsens EO** by amplifying the **TPR disparity**—increasing undertreatment risk for **females** and overtreatment risk for **males**.


In [12]:
from fairlearn.postprocessing import ThresholdOptimizer

# Demographic Parity on top of the CorrelationRemover
post_dp_cr = ThresholdOptimizer(
    estimator=pca_knn,
    constraints="demographic_parity",
    objective="accuracy_score",
    predict_method="predict_proba",
    grid_size=1000,
    prefit=True
)
post_dp_cr.fit(Xtr_fair, y_train, sensitive_features=A_train)  # ideally fit on a validation split
y_dp_cr = post_dp_cr.predict(Xte_fair, sensitive_features=A_test, random_state=42)
m_dp_cr = eval_fairness(y_test, y_dp_cr, A_test)

# Equalized Odds on top of CorrelationRemover
post_eod_cr = ThresholdOptimizer(
    estimator=pca_knn,
    constraints="equalized_odds",
    objective="accuracy_score",
    predict_method="predict_proba",
    grid_size=1000,
    prefit=True
)
post_eod_cr.fit(Xtr_fair, y_train, sensitive_features=A_train)  # ideally fit on a validation split
y_eod_cr = post_eod_cr.predict(Xte_fair, sensitive_features=A_test, random_state=42)
m_eod_cr = eval_fairness(y_test, y_eod_cr, A_test)

# Check changes vs. CR baseline predictions (y_cr)
import numpy as np
print(f"Changed vs CR baseline (DP):  {np.mean(y_dp_cr  != y_cr):.3%}")
print(f"Changed vs CR baseline (eOD): {np.mean(y_eod_cr != y_cr):.3%}")

print("\n=== Post-CR (DP) ===")
print(m_dp_cr["by_group"])
print(f"Accuracy: {m_dp_cr['acc']:.4f} | DP diff: {m_dp_cr['dp']:.4f} | EO diff: {m_dp_cr['eo']:.4f}")

print("\n=== Post-CR (eOD) ===")
print(m_eod_cr["by_group"])
print(f"Accuracy: {m_eod_cr['acc']:.4f} | DP diff: {m_eod_cr['dp']:.4f} | EO diff: {m_eod_cr['eo']:.4f}")


Changed vs CR baseline (DP):  0.000%
Changed vs CR baseline (eOD): 0.000%

=== Post-CR (DP) ===
          TPR    FPR    Recall  SelectionRate  Accuracy
Sex                                                    
0    0.500000  0.125  0.500000       0.184211  0.815789
1    0.802083  0.200  0.802083       0.595890  0.801370
Accuracy: 0.8043 | DP diff: 0.4117 | EO diff: 0.3021

=== Post-CR (eOD) ===
          TPR    FPR    Recall  SelectionRate  Accuracy
Sex                                                    
0    0.500000  0.125  0.500000       0.184211  0.815789
1    0.802083  0.200  0.802083       0.595890  0.801370
Accuracy: 0.8043 | DP diff: 0.4117 | EO diff: 0.3021


**Interpretation:**  
- **No effect:** Post-CR **DP** and **eOD** post-processing changed **0%** — outputs identical to the CR baseline.  
- **Disparities persist:** **DP diff 0.4117** (Female sel. **0.184** vs Male **0.596**), **EO diff 0.3021** (TPR **0.500** vs **0.802**; FPR **0.125** vs **0.200**).  
- **Accuracy:** **0.8043**, unchanged.  

### Bias mitigation comparison (PCA+KNN)  


| Model variant                    | Accuracy | DP diff | EO diff | SelRate S=0 | SelRate S=1 | TPR S=0 | TPR S=1 | FPR S=0 | FPR S=1 | Notes                        |
|----------------------------------|:--------:|:-------:|:-------:|:-----------:|:-----------:|:-------:|:-------:|:-------:|:-------:|------------------------------|
| Baseline (tuned PCA+KNN)         | 0.8315   | 0.4301  | 0.1146  | 0.1316      | 0.5616      | 0.6667  | 0.7813  | 0.0313  | 0.1400  | Reference                    |
| Post-processing (DP constraint)  | 0.8315   | 0.4301  | 0.1146  | 0.1316      | 0.5616      | 0.6667  | 0.7813  | 0.0313  | 0.1400  | **Flips vs baseline: 0%**    |
| Post-processing (EO constraint)  | 0.8315   | 0.4301  | 0.1146  | 0.1316      | 0.5616      | 0.6667  | 0.7813  | 0.0313  | 0.1400  | **Flips vs baseline: 0%**    |
| CorrelationRemover + PCA+KNN     | 0.8043   | 0.4117  | 0.3021  | 0.1842      | 0.5959      | 0.5000  | 0.8021  | 0.1250  | 0.2000  | New baseline after CR        |
| Post-CR (DP constraint)          | 0.8043   | 0.4117  | 0.3021  | 0.1842      | 0.5959      | 0.5000  | 0.8021  | 0.1250  | 0.2000  | **Flips vs CR baseline: 0%** |
| Post-CR (EO constraint)          | 0.8043   | 0.4117  | 0.3021  | 0.1842      | 0.5959      | 0.5000  | 0.8021  | 0.1250  | 0.2000  | **Flips vs CR baseline: 0%** |

**Takeaway:** Post-processing caused **no label changes** in either setting. Applying **CorrelationRemover** slightly reduced **DP** (0.4301→0.4117) but **worsened EO** (0.1146→0.3021) and lowered accuracy (0.8315→0.8043), with males selected far more often than females.

--- 

### Alternative Tuned & Pruned Decision Tree

In [13]:
# Alternative DT tuning focused on higher recall
# Changes vs previous:
#  - Remove calibration (predict uses raw tree probs at 0.5)
#  - Tune class_weight (heavier positive weights allowed)
#  - Broaden depth a bit but keep regularization via min_samples_* and tiny impurity decrease
#  - Prune only with very small ccp_alphas to avoid killing recall

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix
)
import numpy as np

# Simpler-but-expressive trees + tuned class weights
base_dt = DecisionTreeClassifier(random_state=42)

param_grid_simple = {
    "criterion": ["gini", "entropy"],                  # add "log_loss" if your sklearn supports it
    "max_depth": [4, 5, 6, 7, 8, 9, 10],               # a bit deeper to help recall
    "min_samples_split": [5, 10, 20],
    "min_samples_leaf": [1, 2, 4, 6],
    "min_impurity_decrease": [0.0, 1e-4, 1e-3],
    "class_weight": ["balanced", {0:1,1:2}, {0:1,1:3}, {0:1,1:4}],  # stronger push toward positives
}

cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)

grid_simple = GridSearchCV(
    estimator=base_dt,
    param_grid=param_grid_simple,
    cv=cv,
    scoring="recall",      # prioritize sensitivity for class 1
    n_jobs=-1,
    verbose=0,
    refit=True
)
grid_simple.fit(X_train_ready, y_train)

best_params = grid_simple.best_params_
print("Stage A — Best DT params:", best_params)
print("Stage A — Best CV Recall:", round(grid_simple.best_score_, 4))

# Train a zero-pruned model with best params to get the pruning path
dt0 = DecisionTreeClassifier(random_state=42, **best_params, ccp_alpha=0.0).fit(X_train_ready, y_train)


# Stage B — Gentle cost-complexity pruning (favor small alphas)
path = dt0.cost_complexity_pruning_path(X_train_ready, y_train)
ccp_alphas = path.ccp_alphas

# Focus on tiny alphas only + 0.0 to avoid big recall loss
small_slice = ccp_alphas[: min(30, len(ccp_alphas))]  # first 30 values are typically the smallest
candidate_alphas = np.unique(np.r_[0.0, small_slice])

cv_scores = []
for alpha in candidate_alphas:
    dt_alpha = DecisionTreeClassifier(random_state=42, **best_params, ccp_alpha=alpha)
    rec = cross_val_score(dt_alpha, X_train_ready, y_train, cv=cv, scoring="recall", n_jobs=-1).mean()
    cv_scores.append((alpha, rec))

best_alpha, best_cv_recall = max(cv_scores, key=lambda x: x[1])
print(f"Stage B — Best ccp_alpha: {best_alpha:.6f} | CV Recall: {best_cv_recall:.4f}")

alt_best_dt = DecisionTreeClassifier(random_state=42, **best_params, ccp_alpha=best_alpha).fit(X_train_ready, y_train)


# Evaluation
y_pred = alt_best_dt.predict(X_test_ready)               
y_prob = alt_best_dt.predict_proba(X_test_ready)[:, 1]   

evaluate_model(y_test, y_pred, "Alternative Tuned & Pruned Decision Tree")

Stage A — Best DT params: {'class_weight': {0: 1, 1: 4}, 'criterion': 'entropy', 'max_depth': 4, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 4, 'min_samples_split': 5}
Stage A — Best CV Recall: 0.9733
Stage B — Best ccp_alpha: 0.000000 | CV Recall: 0.9733
=== Alternative Tuned & Pruned Decision Tree Evaluation ===
Accuracy : 0.8260869565217391
Precision: 0.7966101694915254
Recall   : 0.9215686274509803
F1 Score : 0.8545454545454545

Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.71      0.78        82
           1       0.80      0.92      0.85       102

    accuracy                           0.83       184
   macro avg       0.84      0.81      0.82       184
weighted avg       0.83      0.83      0.82       184

Confusion Matrix:
 [[58 24]
 [ 8 94]]




### Bias Mitigation DT: Inprocessing - Exponentiated Gradient Reduction

In [15]:
# In-processing mitigation for tuned Decision Tree
from sklearn.base import clone
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds, DemographicParity
from fairlearn.metrics import (
    MetricFrame, true_positive_rate, false_positive_rate, selection_rate,
    demographic_parity_difference, equalized_odds_difference
)
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# 0) Baseline: tuned DT without mitigation (for comparison)
y_pred_dt_base = alt_best_dt.predict(X_test_ready)
m_base = eval_fairness(y_test, y_pred_dt_base, A_test)
print("=== Baseline (Tuned DT) ===")
print(m_base["by_group"])
print(f"Accuracy: {m_base['acc']:.4f} | DP diff: {m_base['dp']:.4f} | EO diff: {m_base['eo']:.4f}")

# 1) Exponentiated Gradient with Equalized Odds
eg_eo = ExponentiatedGradient(
    estimator=clone(alt_best_dt),
    constraints=EqualizedOdds(),
    eps=0.01,
    max_iter=50
)
eg_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_eo = eg_eo.predict(X_test_ready)
m_eo = eval_fairness(y_test, y_pred_eo, A_test)
print("\n=== In-processing: EG (Equalized Odds) ===")
print(m_eo["by_group"])
print(f"Accuracy: {m_eo['acc']:.4f} | DP diff: {m_eo['dp']:.4f} | EO diff: {m_eo['eo']:.4f}")

# 2) Exponentiated Gradient with Demographic Parity
eg_dp = ExponentiatedGradient(
    estimator=clone(alt_best_dt),
    constraints=DemographicParity(),
    eps=0.01,
    max_iter=50
)
eg_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_dp = eg_dp.predict(X_test_ready)
m_dp = eval_fairness(y_test, y_pred_dp, A_test)
print("\n=== In-processing: EG (Demographic Parity) ===")
print(m_dp["by_group"])
print(f"Accuracy: {m_dp['acc']:.4f} | DP diff: {m_dp['dp']:.4f} | EO diff: {m_dp['eo']:.4f}")

# 3) Summary table
summary_dt = pd.DataFrame([
    {"model":"DT Baseline (tuned)", "accuracy":m_base["acc"], "dp_diff":m_base["dp"], "eo_diff":m_base["eo"]},
    {"model":"DT + EG (EO)",        "accuracy":m_eo["acc"],   "dp_diff":m_eo["dp"],   "eo_diff":m_eo["eo"]},
    {"model":"DT + EG (DP)",        "accuracy":m_dp["acc"],   "dp_diff":m_dp["dp"],   "eo_diff":m_dp["eo"]},
]).round(4)
print("\n=== Decision Tree: Baseline vs In-processing (EG) ===")
summary_dt


=== Baseline (Tuned DT) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.28125  0.833333       0.368421  0.736842
1    0.927083  0.30000  0.927083       0.712329  0.849315
Accuracy: 0.8261 | DP diff: 0.3439 | EO diff: 0.0938

=== In-processing: EG (Equalized Odds) ===
          TPR   FPR    Recall  SelectionRate  Accuracy
Sex                                                   
0    0.833333  0.25  0.833333       0.342105  0.763158
1    0.906250  0.28  0.906250       0.691781  0.842466
Accuracy: 0.8261 | DP diff: 0.3497 | EO diff: 0.0729

=== In-processing: EG (Demographic Parity) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.28125  0.833333       0.368421  0.736842
1    0.864583  0.24000  0.864583       0.650685  0.828767
Accuracy: 0.8098 | DP diff: 0.2823 | EO diff: 0.0413

=== Decision Tree: Baseline vs In-pr

,model,accuracy,dp_diff,eo_diff
0,DT Baseline (tuned),0.8261,0.3439,0.0938
1,DT + EG (EO),0.8261,0.3497,0.0729
2,DT + EG (DP),0.8098,0.2823,0.0413


### Bias Mitigation Results: Decision Tree – In-Processing  

#### Metrics Overview

| Model                   | Accuracy | DP diff | EO diff | Notes                                                                 |
|-------------------------|:--------:|:-------:|:-------:|------------------------------------------------------------------------|
| **DT Baseline (tuned)** | 0.8261   | 0.3439  | 0.0938  | Moderate DP disparity; small EO gap. |
| **DT + EG (EO)**        | 0.8261   | 0.3497  | 0.0729  | Accuracy =; **DP worsens slightly**; **EO improves modestly**. |
| **DT + EG (DP)**        | 0.8098   | 0.2823  | 0.0413  | Accuracy ↓ (−0.0163); **DP improves**; **EO improves strongly**. |

---

#### Interpretation
- The **baseline DT** shows a **moderate DP gap (~0.34)** and a **small EO gap (~0.09)**.  
- **EG (EO constraint):** keeps accuracy the same, **reduces EO** (0.0938 → 0.0729), but **slightly increases DP**.  
- **EG (DP constraint):** delivers the **largest fairness gains** — both **DP ↓** (0.344 → 0.282) and **EO ↓** (0.094 → 0.041), though accuracy drops by ~1.6 pp.  

**Conclusion:**  
- **EG (DP)** offers the **best fairness improvements overall** (both DP and EO reduced) with only a small accuracy cost.  
- **EG (EO)** provides a minor EO benefit without hurting accuracy, but DP slightly worsens.  
- The **baseline** is already relatively fair on EO; mitigation mainly helps if **DP reduction** is prioritized.  

---

#### Bias Mitigation DT: In-processing: GridSearch Reduction

In [16]:
from sklearn.base import clone
from fairlearn.reductions import GridSearch, EqualizedOdds, DemographicParity

# 1) GridSearch with Equalized Odds
gs_eo = GridSearch(
    estimator=clone(alt_best_dt),              # unfitted clone of tuned DT
    constraints=EqualizedOdds(),            # EO constraint
    selection_rule="tradeoff_optimization", 
    constraint_weight=0.5,                  
    grid_size=15,                           
)
gs_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_gs_eo = gs_eo.predict(X_test_ready)
m_gs_eo = eval_fairness(y_test, y_pred_gs_eo, A_test)
print("\n=== In-processing: GridSearch (Equalized Odds) ===")
print(m_gs_eo["by_group"])
print(f"Accuracy: {m_gs_eo['acc']:.4f} | DP diff: {m_gs_eo['dp']:.4f} | EO diff: {m_gs_eo['eo']:.4f}")

# 2) GridSearch with Demographic Parity
gs_dp = GridSearch(
    estimator=clone(alt_best_dt),
    constraints=DemographicParity(),
    selection_rule="tradeoff_optimization",
    constraint_weight=0.5,
    grid_size=15,
)
gs_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_gs_dp = gs_dp.predict(X_test_ready)
m_gs_dp = eval_fairness(y_test, y_pred_gs_dp, A_test)
print("\n=== In-processing: GridSearch (Demographic Parity) ===")
print(m_gs_dp["by_group"])
print(f"Accuracy: {m_gs_dp['acc']:.4f} | DP diff: {m_gs_dp['dp']:.4f} | EO diff: {m_gs_dp['eo']:.4f}")

# 3) Compare with your existing runs
summary_dt = pd.concat([
    summary_dt,  
    pd.DataFrame([
        {"model":"DT + GS (EO)", "accuracy":m_gs_eo["acc"], "dp_diff":m_gs_eo["dp"], "eo_diff":m_gs_eo["eo"]},
        {"model":"DT + GS (DP)", "accuracy":m_gs_dp["acc"], "dp_diff":m_gs_dp["dp"], "eo_diff":m_gs_dp["eo"]},
    ]).round(4)
], ignore_index=True)
print("\n=== Decision Tree: Baseline vs EG vs GS ===")
summary_dt


=== In-processing: GridSearch (Equalized Odds) ===
          TPR   FPR    Recall  SelectionRate  Accuracy
Sex                                                   
0    0.666667  0.25  0.666667       0.315789  0.736842
1    0.718750  0.24  0.718750       0.554795  0.732877
Accuracy: 0.7337 | DP diff: 0.2390 | EO diff: 0.0521

=== In-processing: GridSearch (Demographic Parity) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.21875  0.833333       0.315789  0.789474
1    0.916667  0.40000  0.916667       0.739726  0.808219
Accuracy: 0.8043 | DP diff: 0.4239 | EO diff: 0.1813

=== Decision Tree: Baseline vs EG vs GS ===


,model,accuracy,dp_diff,eo_diff
0,DT Baseline (tuned),0.8261,0.3439,0.0938
1,DT + EG (EO),0.8261,0.3497,0.0729
2,DT + EG (DP),0.8098,0.2823,0.0413
3,DT + GS (EO),0.7337,0.2390,0.0521
4,DT + GS (DP),0.8043,0.4239,0.1813


### Decision Tree — In-Processing: EG vs. GridSearch (EO & DP)

#### Summary of results
| Model                    | Accuracy | DP diff | EO diff | Notes |
|--------------------------|:--------:|:-------:|:-------:|------|
| **DT Baseline (tuned)**  | 0.8261   | 0.3439  | 0.0938  | Moderate DP gap; small EO gap. |
| **DT + EG (EO)**         | 0.8261   | 0.3497  | 0.0729  | **Acc =** baseline; **EO improves modestly**; DP slightly worse. |
| **DT + EG (DP)**         | 0.8098   | 0.2823  | 0.0413  | **Acc ↓ ~1.6 pp**; **both DP and EO improve** vs baseline. |
| **DT + GS (EO)**         | 0.7337   | **0.2390** | 0.0521 | **Lowest DP** & low EO, but **accuracy drops sharply**. |
| **DT + GS (DP)**         | 0.8043   | 0.4239  | 0.1813  | **Acc ↓**; **DP and EO worsen**. |

---

#### Interpretation
- **EG (DP)** gives the **cleanest fairness gains** here: it **reduces both DP and EO** (0.3439→0.2823; 0.0938→0.0413) with a **small accuracy trade-off**.
- **EG (EO)** delivers a **modest EO improvement** at **no accuracy cost**, but nudges DP upward a bit.
- **GS (EO)** attains the **lowest DP** and a low EO but at the expense of a **large accuracy drop (to 0.7337)**—not a practical choice.
- **GS (DP)** is **counterproductive**, degrading both fairness metrics and accuracy.

**Summary:**  
If you can tolerate a slight accuracy decrease, **DT + EG(DP)** is the best fairness–utility trade-off. If accuracy must remain unchanged, **DT + EG(EO)** offers a smaller EO gain with near-baseline performance. Avoid **GS(DP)**, and treat **GS(EO)** as impractical due to its accuracy hit.

----

#### Bias Mitigation DT: Post-processing: Threshold Optimizer 

In [17]:
from fairlearn.postprocessing import ThresholdOptimizer

#Baseline for mitigation: fixed tuned DT
alt_best_dt.fit(X_train_ready, y_train)
y_base = alt_best_dt.predict(X_test_ready)
m_base = eval_fairness(y_test, y_base, A_test)
print("=== Baseline (tuned DT) ===")
print(m_base["by_group"])
print(f"Accuracy: {m_base['acc']:.4f} | DP diff: {m_base['dp']:.4f} | EO diff: {m_base['eo']:.4f}")

#Post-processing: Equalized Odds
post_eo = ThresholdOptimizer(
    estimator=alt_best_dt,
    constraints="equalized_odds",
    predict_method="predict_proba",   
    grid_size=200,
    flip=True
)
post_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_eo = post_eo.predict(X_test_ready, sensitive_features=A_test, random_state=42)
m_eo = eval_fairness(y_test, y_eo, A_test)
print("\n=== Post-processing (Equalized Odds) ===")
print(m_eo["by_group"])
print(f"Accuracy: {m_eo['acc']:.4f} | DP diff: {m_eo['dp']:.4f} | EO diff: {m_eo['eo']:.4f}")

# Post-processing: Demographic Parity
post_dp = ThresholdOptimizer(
    estimator=alt_best_dt,
    constraints="demographic_parity",
    predict_method="predict_proba",
    grid_size=200,
    flip=True
)
post_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_dp = post_dp.predict(X_test_ready, sensitive_features=A_test, random_state=42)
m_dp = eval_fairness(y_test, y_dp, A_test)
print("\n=== Post-processing (Demographic Parity) ===")
print(m_dp["by_group"])
print(f"Accuracy: {m_dp['acc']:.4f} | DP diff: {m_dp['dp']:.4f} | EO diff: {m_dp['eo']:.4f}")

# create summary table 
summary = pd.DataFrame([
    {"model":"DT Baseline (tuned)", "accuracy":m_base["acc"], "dp_diff":m_base["dp"], "eo_diff":m_base["eo"]},
    {"model":"DT + Post (EO)",      "accuracy":m_eo["acc"],   "dp_diff":m_eo["dp"],   "eo_diff":m_eo["eo"]},
    {"model":"DT + Post (DP)",      "accuracy":m_dp["acc"],   "dp_diff":m_dp["dp"],   "eo_diff":m_dp["eo"]},
]).round(4)
print("\n=== Decision Tree: Baseline vs Post-processing ===")
summary

=== Baseline (tuned DT) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.28125  0.833333       0.368421  0.736842
1    0.927083  0.30000  0.927083       0.712329  0.849315
Accuracy: 0.8261 | DP diff: 0.3439 | EO diff: 0.0938

=== Post-processing (Equalized Odds) ===
          TPR    FPR    Recall  SelectionRate  Accuracy
Sex                                                    
0    0.333333  0.125  0.333333       0.157895  0.789474
1    0.625000  0.160  0.625000       0.465753  0.698630
Accuracy: 0.7174 | DP diff: 0.3079 | EO diff: 0.2917

=== Post-processing (Demographic Parity) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.666667  0.15625  0.666667       0.236842  0.815789
1    0.822917  0.18000  0.822917       0.602740  0.821918
Accuracy: 0.8207 | DP diff: 0.3659 | EO diff: 0.1562

=== Decision Tree: Baseline vs Post-

,model,accuracy,dp_diff,eo_diff
0,DT Baseline (tuned),0.8261,0.3439,0.0938
1,DT + Post (EO),0.7174,0.3079,0.2917
2,DT + Post (DP),0.8207,0.3659,0.1562


### Decision Tree — Post- vs In-Processing 

#### Combined Results

| Model / Method          | Accuracy | DP diff | EO diff | Notes (vs. baseline 0.8261 / 0.3439 / 0.0938) |
|-------------------------|:--------:|:-------:|:-------:|-----------------------------------------------|
| **Baseline (Tuned DT)** | 0.8261   | 0.3439  | 0.0938  | Reference                                      |
| **Post (EO)**           | 0.7174   | 0.3079  | 0.2917  | Accuracy ↓ sharply (−0.11); DP ↓ slightly; **EO worsens strongly** |
| **Post (DP)**           | 0.8207   | 0.3659  | 0.1562  | Accuracy ≈ baseline; DP ↑ slightly; **EO worsens** |
| **EG (EO)**             | 0.8261   | 0.3497  | 0.0729  | Accuracy = baseline; DP ↑ slightly; **EO improves modestly** |
| **EG (DP)**             | 0.8098   | 0.2823  | 0.0413  | Accuracy ↓ (~−1.6 pp); **both DP and EO improve** vs baseline |
| **GS (EO)**             | 0.7337   | **0.2390** | 0.0521 | Accuracy ↓ heavily; **lowest DP and EO**, but poor utility overall |
| **GS (DP)**             | 0.8043   | 0.4239  | 0.1813  | Accuracy ↓ moderately; DP & EO both worsen vs baseline |

---

#### Interpretation
- **Baseline DT** shows a **moderate DP gap (~0.34)** and a **small EO gap (~0.09)**.  
- **Post (EO):** slight DP gain but EO disparity more than triples, accuracy collapses → **not usable**.  
- **Post (DP):** keeps accuracy near baseline but worsens both DP and EO → **ineffective**.  
- **EG (EO):** yields a **modest EO improvement** with baseline accuracy; DP worsens slightly.  
- **EG (DP):** actually improves both DP and EO but at a small accuracy cost (~−1.6 pp).  
- **GS (EO):** achieves the **lowest DP and EO** but accuracy falls too much (0.7337) → impractical.  
- **GS (DP):** harms both fairness metrics and reduces accuracy.

---

**Conclusion:**  
- If **EO parity** is the main target, **EG (DP)** is surprisingly the best compromise: **both DP and EO improve**, with only a small accuracy drop.  
- **EG (EO)** is safer if accuracy must remain at baseline, offering modest EO gains.  
- **Post-processing methods** do not yield useful fairness–utility trade-offs in this setup.  
- **GS (EO)** looks good on fairness metrics but its accuracy collapse rules it out for clinical use.  
- Overall, **baseline DT remains competitive**, with only incremental improvements possible through EG-based approaches.  

---

### Ensemble Model - Random Forest (RF)

In [18]:
# Random Forest: hyperparameter tuning
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

# 1) GridSearchCV over impactful RF params
rf = RandomForestClassifier(random_state=42)

param_grid = {
    "n_estimators": [200, 400, 600],
    "max_depth": [None, 8, 12, 16],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", 0.8],  # 0.8 = 80% of features
    "class_weight": [None, "balanced"]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=cv,
    scoring="recall",          
    n_jobs=-1,
    verbose=1,
    refit=True
)

grid.fit(X_train_ready, y_train)
best_rf = grid.best_estimator_
print("Best RF params:", grid.best_params_)
print("Best CV F1:", grid.best_score_)

# 2) Evaluate best RF 
y_pred = best_rf.predict(X_test_ready)
y_prob = best_rf.predict_proba(X_test_ready)[:, 1]
evaluate_model(y_test, y_pred, "Tuned Random Forest")

Fitting 5 folds for each of 648 candidates, totalling 3240 fits
Best RF params: {'class_weight': None, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 400}
Best CV F1: 0.9566666666666667
=== Tuned Random Forest Evaluation ===
Accuracy : 0.842391304347826
Precision: 0.9010989010989011
Recall   : 0.803921568627451
F1 Score : 0.8497409326424871

Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.89      0.83        82
           1       0.90      0.80      0.85       102

    accuracy                           0.84       184
   macro avg       0.84      0.85      0.84       184
weighted avg       0.85      0.84      0.84       184

Confusion Matrix:
 [[73  9]
 [20 82]]




### Bias Mitgation RF: In-processing: Exponentiated Gradient 

In [19]:
# 0) Baseline Random Forest
best_rf = RandomForestClassifier(random_state=42)
best_rf.fit(X_train_ready, y_train)

y_pred_rf_base = best_rf.predict(X_test_ready)
m_rf_base = eval_fairness(y_test, y_pred_rf_base, A_test)

print("=== Baseline (Random Forest) ===")
print(m_rf_base["by_group"])
print(f"Accuracy: {m_rf_base['acc']:.4f} | DP diff: {m_rf_base['dp']:.4f} | EO diff: {m_rf_base['eo']:.4f}")

#1) EG with Equalized Odds
eg_eo_rf = ExponentiatedGradient(
    estimator=clone(best_rf),
    constraints=EqualizedOdds(),
    eps=0.01,
    max_iter=50
)
eg_eo_rf.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_rf_eo = eg_eo_rf.predict(X_test_ready, random_state=42)
m_rf_eo = eval_fairness(y_test, y_pred_rf_eo, A_test)

print("\n=== In-processing RF: EG (Equalized Odds) ===")
print(m_rf_eo["by_group"])
print(f"Accuracy: {m_rf_eo['acc']:.4f} | DP diff: {m_rf_eo['dp']:.4f} | EO diff: {m_rf_eo['eo']:.4f}")

# 2) EG with Demographic Parity 
eg_dp_rf = ExponentiatedGradient(
    estimator=clone(best_rf),
    constraints=DemographicParity(),
    eps=0.01,
    max_iter=50
)
eg_dp_rf.fit(X_train_ready, y_train, sensitive_features=A_train)
y_pred_rf_dp = eg_dp_rf.predict(X_test_ready, random_state=42)
m_rf_dp = eval_fairness(y_test, y_pred_rf_dp, A_test)

print("\n=== In-processing RF: EG (Demographic Parity) ===")
print(m_rf_dp["by_group"])
print(f"Accuracy: {m_rf_dp['acc']:.4f} | DP diff: {m_rf_dp['dp']:.4f} | EO diff: {m_rf_dp['eo']:.4f}")

# 3) Summary Table 
summary_rf = pd.DataFrame([
    {"model":"RF Baseline",      "accuracy":m_rf_base["acc"], "dp_diff":m_rf_base["dp"], "eo_diff":m_rf_base["eo"]},
    {"model":"RF + EG (EO)",     "accuracy":m_rf_eo["acc"],   "dp_diff":m_rf_eo["dp"],   "eo_diff":m_rf_eo["eo"]},
    {"model":"RF + EG (DP)",     "accuracy":m_rf_dp["acc"],   "dp_diff":m_rf_dp["dp"],   "eo_diff":m_rf_dp["eo"]},
]).round(4)

print("\n=== Random Forest: Baseline vs In-processing (EG) ===")
print(summary_rf)

=== Baseline (Random Forest) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.03125  0.833333       0.157895  0.947368
1    0.791667  0.14000  0.791667       0.568493  0.815068
Accuracy: 0.8424 | DP diff: 0.4106 | EO diff: 0.1088

=== In-processing RF: EG (Equalized Odds) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.03125  0.833333       0.157895  0.947368
1    0.791667  0.14000  0.791667       0.568493  0.815068
Accuracy: 0.8424 | DP diff: 0.4106 | EO diff: 0.1088

=== In-processing RF: EG (Demographic Parity) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.03125  0.833333       0.157895  0.947368
1    0.791667  0.14000  0.791667       0.568493  0.815068
Accuracy: 0.8424 | DP diff: 0.4106 | EO diff: 0.1088

=== Random Fo

## Random Forest Bias Mitigation Results  


### Summary

| Model            | Accuracy | DP diff | EO diff | Interpretation                                 |
|------------------|:--------:|:-------:|:-------:|-----------------------------------------------|
| **RF Baseline**  | 0.8424   | 0.4106  | 0.1088  | Good accuracy; **large DP gap** persists.      |
| **RF + EG (EO)** | 0.8424   | 0.4106  | 0.1088  | **No change** vs baseline → EO constraint had no effect. |
| **RF + EG (DP)** | 0.8424   | 0.4106  | 0.1088  | **No change** vs baseline → DP constraint had no effect. |

### Key Points
- Selection rates: **0.158 (F)** vs **0.568 (M)** → ~**3.6×** higher for males (**DP 0.4106**).
- Error rates: **TPR** 0.833 (F) vs 0.792 (M); **FPR** 0.031 (F) vs 0.140 (M) → **EO 0.1088**.
- In-processing EG (EO/DP) produced **0% movement**—likely constraints not binding or the optimizer chose the baseline point on the fairness–accuracy frontier.

---

### Bias Mitigation: RF: In-processing: Grid Search

In [20]:
from sklearn.base import clone
from fairlearn.reductions import GridSearch, EqualizedOdds, DemographicParity

weights = [0.0, 0.25, 0.5, 0.75, 1.0]   # 0.0 = accuracy-first, 1.0 = fairness-first
grid = 50                               

rows = []

#Equalized Odds sweep
for w in weights:
    gs_eo_rf = GridSearch(
        estimator=clone(best_rf),                 
        constraints=EqualizedOdds(),
        selection_rule="tradeoff_optimization",
        constraint_weight=w,
        grid_size=grid
    )
    gs_eo_rf.fit(X_train_ready, y_train, sensitive_features=A_train)
    # Some versions accept random_state in predict; if yours doesn't, seed numpy before predicting
    try:
        y_hat = gs_eo_rf.predict(X_test_ready, random_state=42)
    except TypeError:
        import numpy as np, random
        np.random.seed(42); random.seed(42)
        y_hat = gs_eo_rf.predict(X_test_ready)
    m = eval_fairness(y_test, y_hat, A_test)
    rows.append({"method":"RF + GS (EO)", "weight": w, "acc": m["acc"], "dp": m["dp"], "eo": m["eo"]})

# Demographic Parity sweep
for w in weights:
    gs_dp_rf = GridSearch(
        estimator=clone(best_rf),
        constraints=DemographicParity(),
        selection_rule="tradeoff_optimization",
        constraint_weight=w,
        grid_size=grid
    )
    gs_dp_rf.fit(X_train_ready, y_train, sensitive_features=A_train)
    try:
        y_hat = gs_dp_rf.predict(X_test_ready, random_state=42)
    except TypeError:
        import numpy as np, random
        np.random.seed(42); random.seed(42)
        y_hat = gs_dp_rf.predict(X_test_ready)
    m = eval_fairness(y_test, y_hat, A_test)
    rows.append({"method":"RF + GS (DP)", "weight": w, "acc": m["acc"], "dp": m["dp"], "eo": m["eo"]})

df_gs = pd.DataFrame(rows).sort_values(["method","weight"])
print(df_gs)

         method  weight       acc        dp    eo
5  RF + GS (DP)    0.00  0.836957  0.470079  0.16
6  RF + GS (DP)    0.25  0.836957  0.470079  0.16
7  RF + GS (DP)    0.50  0.836957  0.470079  0.16
8  RF + GS (DP)    0.75  0.836957  0.470079  0.16
9  RF + GS (DP)    1.00  0.836957  0.470079  0.16
0  RF + GS (EO)    0.00  0.831522  0.476929  0.18
1  RF + GS (EO)    0.25  0.831522  0.476929  0.18
2  RF + GS (EO)    0.50  0.831522  0.476929  0.18
3  RF + GS (EO)    0.75  0.831522  0.476929  0.18
4  RF + GS (EO)    1.00  0.831522  0.476929  0.18


**Interpretation (RF + GridSearch)**

- **No movement across weights:** For both **DP** and **EO** constraints, changing the weight from **0 → 1** yields the **same model** and metrics.
- **Worse than RF baseline:**  
  - **Accuracy:** 0.837 (DP) / 0.832 (EO) vs baseline 0.842 (↓ ~0.5–1.0 pts)  
  - **DP diff:** ~**0.47** vs baseline **0.41** (↑ → fairness gap widened)  
  - **EO diff:** **0.16–0.18** vs baseline **0.109** (↑ → error-rate disparity worsened)

**Takeaway:** GridSearch did **not** explore the fairness–accuracy frontier as expected. Instead, it converged to a single corner solution that is **less fair and slightly less accurate** than the baseline RF.

In [21]:
# Inspect how many distinct models GridSearch actually produced
len(gs_eo_rf.predictors_), len(gs_dp_rf.predictors_)

# See the spread across the frontier (test metrics for each predictor)
def eval_frontier(gs, X, y, A):
    rows=[]
    for i, clf in enumerate(gs.predictors_):
        yhat = clf.predict(X)
        m = eval_fairness(y, yhat, A)
        rows.append({"i": i, "acc": m["acc"], "dp": m["dp"], "eo": m["eo"]})
    return pd.DataFrame(rows)

print(eval_frontier(gs_eo_rf, X_test_ready, y_test, A_test))
print(eval_frontier(gs_dp_rf, X_test_ready, y_test, A_test))


     i       acc        dp        eo
0    0  0.456522  0.052632  0.333333
1    1  0.456522  0.052632  0.333333
2    2  0.456522  0.052632  0.333333
3    3  0.456522  0.052632  0.333333
4    4  0.456522  0.052632  0.333333
5    5  0.456522  0.052632  0.333333
6    6  0.456522  0.052632  0.333333
7    7  0.456522  0.052632  0.333333
8    8  0.347826  0.297765  0.760000
9    9  0.456522  0.052632  0.333333
10  10  0.456522  0.052632  0.333333
11  11  0.461957  0.078947  0.500000
12  12  0.456522  0.052632  0.333333
13  13  0.456522  0.052632  0.333333
14  14  0.375000  0.285148  0.740000
15  15  0.347826  0.325162  0.800000
16  16  0.679348  0.363014  0.760000
17  17  0.831522  0.476929  0.180000
18  18  0.847826  0.497477  0.180000
19  19  0.820652  0.410598  0.148750
20  20  0.836957  0.397981  0.097500
21  21  0.842391  0.424297  0.128750
22  22  0.695652  0.736842  0.843750
23  23  0.717391  0.842105  0.968750
24  24  0.717391  0.842105  0.968750
25  25  0.673913  0.383562  0.780000
2

### Interpretation

**What’s in the tables:** Each index `i` corresponds to one GridSearch candidate along the fairness–accuracy frontier.  
- Many low-index models (e.g., `i=0–12`) collapse to weak classifiers (Acc ≈0.4565, EO ≈0.333) → not useful.  
- Mid-range indices (`i=38–49`) contain the strongest trade-offs.  

#### Strong candidates
- **Best overall (↑Acc, ↓DP, ↓EO):**  
  - `i=40` → **Acc 0.8478**, **DP 0.4037**, **EO 0.0888**.  
  - Improves both fairness metrics compared to baseline while slightly increasing accuracy.

- **Lowest EO gap at near-baseline accuracy:**  
  - `i=29` → **Acc 0.8424**, **DP 0.3911**, **EO 0.0775**.  
  - Strong option if minimizing error-rate disparity is the main goal.

- **Lower DP with modest accuracy trade-off:**  
  - `i=24` (2nd table) → **Acc 0.8261**, **DP 0.3569**, **EO 0.0729**.  
  - `i=47` (1st table) → **Acc 0.8261**, **DP 0.3706**, **EO 0.0775**.  
  - Both reduce selection-rate gap more aggressively, but cost ~1.5–2 points of accuracy.

- **Balanced trade-off (both fairness metrics improved, small Acc dip):**  
  - `i=31` (1st table) → **Acc 0.8370**, **DP 0.3843**, **EO 0.0775**.  
  - A middle ground if you want improvements in DP & EO without losing much accuracy.

**Takeaway:**  
- If **maximizing overall performance with fairness gains** → choose **`i=40`**.  
- If **error-rate parity (EO)** is top priority → choose **`i=29`**.  
- If **reducing demographic disparity (DP)** is more important, and you can accept a modest accuracy loss → choose **`i=24`** or **`i=47`**.  
- **`i=31`** offers a well-balanced compromise.


---

In [22]:
# Show results for the specific frontier models 
# for both RF GridSearch runs (EO- and DP-constrained).

import pandas as pd

indices = [40,29,31]

def eval_selected(gs, label):
    rows = []
    n = len(gs.predictors_)
    print(f"\n=== {label}: {n} frontier candidates ===")
    for i in indices:
        if i >= n:
            print(f"[{label}] Skipping i={i} (only {n} candidates).")
            continue
        clf = gs.predictors_[i]
        y_hat = clf.predict(X_test_ready)
        m = eval_fairness(y_test, y_hat, A_test)
        rows.append({"i": i, "accuracy": m["acc"], "dp_diff": m["dp"], "eo_diff": m["eo"]})

        # Per-group breakdown for this model
        print(f"\n[{label}] i={i}")
        print(m["by_group"])
        print(f"Accuracy: {m['acc']:.4f} | DP diff: {m['dp']:.4f} | EO diff: {m['eo']:.4f}")

    if rows:
        df = pd.DataFrame(rows).sort_values("i").round(4)
        print(f"\n--- Summary ({label}) ---")
        print(df)

# Evaluate selected indices for both EO and DP GridSearch objects
eval_selected(gs_eo_rf, "RF + GS (EO)")
eval_selected(gs_dp_rf, "RF + GS (DP)")


=== RF + GS (EO): 50 frontier candidates ===

[RF + GS (EO)] i=40
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.03125  0.833333       0.157895  0.947368
1    0.791667  0.12000  0.791667       0.561644  0.821918
Accuracy: 0.8478 | DP diff: 0.4037 | EO diff: 0.0887

[RF + GS (EO)] i=29
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.833333  0.0625  0.833333       0.184211  0.921053
1    0.802083  0.1400  0.802083       0.575342  0.821918
Accuracy: 0.8424 | DP diff: 0.3911 | EO diff: 0.0775

[RF + GS (EO)] i=31
          TPR     FPR    Recall  SelectionRate  Accuracy
Sex                                                     
0    0.833333  0.0625  0.833333       0.184211  0.921053
1    0.791667  0.1400  0.791667       0.568493  0.815068
Accuracy: 0.8370 | DP diff: 0.3843 | EO diff: 0.0775

--- Summary (RF + GS (EO)) ---
    i  accur

### Interpretation (RF + GridSearch)

#### Equalized Odds (EO)
- **Best overall (accuracy + fairness):**  
  - `i=40` → **Acc 0.8478**, **DP 0.4037**, **EO 0.0888**.  
    - Slight accuracy gain over baseline, EO gap reduced, DP gap still moderate.  
- **Lowest EO gap (with small accuracy trade-off):**  
  - `i=29` → **Acc 0.8424**, **DP 0.3911**, **EO 0.0775**.  
    - Very good fairness improvement (EO close to zero), accuracy only slightly below baseline.  
- **Balanced candidate:**  
  - `i=31` → **Acc 0.8370**, **DP 0.3843**, **EO 0.0775**.  
    - Both fairness metrics improved, but accuracy ~0.5% lower.

**Takeaway (EO):**  
- If **accuracy + stability** matter most → pick `i=40`.  
- If **minimizing EO disparity** is the top goal → pick `i=29`.  
- If a **balanced trade-off** is desired → `i=31` is solid.

---

#### Demographic Parity (DP)
- **Unstable/poor solution:**  
  - `i=40` → **Acc 0.7065**, **DP 0.8421**, **EO 0.9375**.  
    - Very low accuracy and extreme fairness gaps — discard.  
- **High accuracy but fairness not great:**  
  - `i=29` → **Acc 0.8478**, **DP 0.4311**, **EO 0.1288**.  
    - Accuracy similar to EO frontier, but fairness weaker (DP gap larger, EO gap higher).  
- **Best DP candidate:**  
  - `i=31` → **Acc 0.8315**, **DP 0.3911**, **EO 0.0975**.  
    - Both fairness metrics improved with only a small drop in accuracy.

**Takeaway (DP):**  
- Ignore unstable `i=40`.  
- `i=29` achieves high accuracy but fairness is weaker.  
- **Best balance is `i=31`**, which delivers meaningful fairness gains with a modest accuracy trade-off.

---

### Overall Summary
- **EO-constrained GridSearch** provided more reliable frontier candidates than **DP-constrained**.  
- **Recommended picks:**  
  - `i=40` (EO) → highest accuracy, fairness improved.  
  - `i=29` (EO) → lowest EO disparity, accuracy near baseline.  
  - `i=31` (DP) → best DP-driven balance, but accuracy slightly lower.

---

### Bias Mitigation RF: Post-processing: Threshold Optimizer

In [23]:
from fairlearn.postprocessing import ThresholdOptimizer

# 0) Baseline RF 
best_rf.fit(X_train_ready, y_train)
y_rf_base = best_rf.predict(X_test_ready)
m_rf_base = eval_fairness(y_test, y_rf_base, A_test)

print("=== Baseline (Random Forest) ===")
print(m_rf_base["by_group"])
print(f"Accuracy: {m_rf_base['acc']:.4f} | DP diff: {m_rf_base['dp']:.4f} | EO diff: {m_rf_base['eo']:.4f}")

# 1) Post-processing: Equalized Odds 
post_rf_eo = ThresholdOptimizer(
    estimator=best_rf,
    constraints="equalized_odds",
    predict_method="predict_proba",   
    grid_size=200,
    flip=True
)
post_rf_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_rf_eo = post_rf_eo.predict(X_test_ready, sensitive_features=A_test)
m_rf_eo = eval_fairness(y_test, y_rf_eo, A_test)

print("\n=== RF + Post-processing (Equalized Odds) ===")
print(m_rf_eo["by_group"])
print(f"Accuracy: {m_rf_eo['acc']:.4f} | DP diff: {m_rf_eo['dp']:.4f} | EO diff: {m_rf_eo['eo']:.4f}")

# 2) Post-processing: Demographic Parity 
post_rf_dp = ThresholdOptimizer(
    estimator=best_rf,
    constraints="demographic_parity",
    predict_method="predict_proba",
    grid_size=200,
    flip=True
)
post_rf_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_rf_dp = post_rf_dp.predict(X_test_ready, sensitive_features=A_test)
m_rf_dp = eval_fairness(y_test, y_rf_dp, A_test)

print("\n=== RF + Post-processing (Demographic Parity) ===")
print(m_rf_dp["by_group"])
print(f"Accuracy: {m_rf_dp['acc']:.4f} | DP diff: {m_rf_dp['dp']:.4f} | EO diff: {m_rf_dp['eo']:.4f}")

#3) Summary Table
summary_rf_post = pd.DataFrame([
    {"model":"RF Baseline",       "accuracy":m_rf_base["acc"], "dp_diff":m_rf_base["dp"], "eo_diff":m_rf_base["eo"]},
    {"model":"RF + Post (EO)",    "accuracy":m_rf_eo["acc"],   "dp_diff":m_rf_eo["dp"],   "eo_diff":m_rf_eo["eo"]},
    {"model":"RF + Post (DP)",    "accuracy":m_rf_dp["acc"],   "dp_diff":m_rf_dp["dp"],   "eo_diff":m_rf_dp["eo"]},
]).round(4)

print("\n=== Random Forest: Baseline vs Post-processing ===")
print(summary_rf_post)

=== Baseline (Random Forest) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.833333  0.03125  0.833333       0.157895  0.947368
1    0.791667  0.14000  0.791667       0.568493  0.815068
Accuracy: 0.8424 | DP diff: 0.4106 | EO diff: 0.1088

=== RF + Post-processing (Equalized Odds) ===
          TPR   FPR    Recall  SelectionRate  Accuracy
Sex                                                   
0    0.666667  0.00  0.666667       0.105263  0.947368
1    0.791667  0.14  0.791667       0.568493  0.815068
Accuracy: 0.8424 | DP diff: 0.4632 | EO diff: 0.1400

=== RF + Post-processing (Demographic Parity) ===
          TPR   FPR    Recall  SelectionRate  Accuracy
Sex                                                   
0    0.666667  0.00  0.666667       0.105263  0.947368
1    0.791667  0.14  0.791667       0.568493  0.815068
Accuracy: 0.8424 | DP diff: 0.4632 | EO diff: 0.1400

=== Random Forest: Baseline vs Post-p

### Random Forest Bias Mitigation: Post-processing: Threshold Optimizer

## Summary

| Model               | Accuracy | DP diff | EO diff | Interpretation                                    |
|---------------------|:--------:|:-------:|:-------:|---------------------------------------------------|
| **RF Baseline**     | 0.8424   | 0.4106  | 0.1088  | Solid accuracy; sizeable DP gap; moderate EO gap. |
| **RF + Post (EO)**  | 0.8424   | 0.4632  | 0.1400  | Accuracy unchanged; **DP worsens**; **EO worsens**. |
| **RF + Post (DP)**  | 0.8424   | 0.4632  | 0.1400  | Identical to EO result → no fairness gain.        |

### Summary:
- **Accuracy** remained flat (0.8424) across baseline and post-processing.  
- **Demographic Parity difference** increased from **0.4106 → 0.4632** → gap widened.  
- **Equalized Odds difference** also worsened from **0.1088 → 0.1400**.  
- Both EO and DP post-processing collapsed to the **same adjusted solution**, offering **no fairness improvement**.

**Takeaway:** For RF, this post-processing configuration **failed to improve fairness** and instead made disparities worse while leaving accuracy unchanged.  

---

### Deep Learning - Multi-layer Perceptron

In [24]:
#import required library 
from sklearn.neural_network import MLPClassifier

In [25]:
#Adam + Early Stopping 
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix

adammlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),   # slightly smaller/deeper can help
    activation='relu',
    solver='adam',
    learning_rate_init=1e-3,       # smaller step can stabilize
    alpha=1e-3,                    # L2 regularization to reduce overfitting
    batch_size=32,
    max_iter=1000,                 # increased max_iter
    early_stopping=True,           # use a validation split internally
    validation_fraction=0.15,
    n_iter_no_change=25,          
    tol=1e-4,
    random_state=42
)

adammlp.fit(X_train_ready, y_train)  
y_pred_mlp = adammlp.predict(X_test_ready)                     
y_prob_mlp = adammlp.predict_proba(X_test_ready)[:, 1]         

evaluate_model(y_test, y_pred_mlp, "(Adam + EarlyStopping)")

=== (Adam + EarlyStopping) Evaluation ===
Accuracy : 0.8206521739130435
Precision: 0.8709677419354839
Recall   : 0.7941176470588235
F1 Score : 0.8307692307692308

Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.85      0.81        82
           1       0.87      0.79      0.83       102

    accuracy                           0.82       184
   macro avg       0.82      0.82      0.82       184
weighted avg       0.83      0.82      0.82       184

Confusion Matrix:
 [[70 12]
 [21 81]]




### Bias mitigation MLP: Inprocessing: Exponentiated Gradient 

In [26]:
from sklearn.neural_network import MLPClassifier
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds, DemographicParity
from sklearn.base import clone
import pandas as pd

# 0) Baseline MLP (seeded for reproducibility)
mlp = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    solver='adam',
    learning_rate_init=1e-3,
    alpha=1e-3,
    batch_size=32,
    max_iter=1000,
    early_stopping=True,
    validation_fraction=0.15,
    n_iter_no_change=25,
    tol=1e-4,
    random_state=42
)

mlp.fit(X_train_ready, y_train)

y_pred_mlp_base = mlp.predict(X_test_ready)
m_mlp_base = eval_fairness(y_test, y_pred_mlp_base, A_test)

print("=== Baseline (MLP) ===")
print(m_mlp_base["by_group"])
print(f"Accuracy: {m_mlp_base['acc']:.4f} | DP diff: {m_mlp_base['dp']:.4f} | EO diff: {m_mlp_base['eo']:.4f}")

# 1) EG with Equalized Odds
eg_eo_mlp = ExponentiatedGradient(
    estimator=clone(mlp),   # inherits random_state=42
    constraints=EqualizedOdds(),
    eps=0.01,
    max_iter=50
)
eg_eo_mlp.fit(X_train_ready, y_train, sensitive_features=A_train)

# Prefer predict(..., random_state=42) if supported; otherwise fall back without global seeds
try:
    y_pred_mlp_eo = eg_eo_mlp.predict(X_test_ready, random_state=42)
except TypeError:
    y_pred_mlp_eo = eg_eo_mlp.predict(X_test_ready)

m_mlp_eo = eval_fairness(y_test, y_pred_mlp_eo, A_test)

print("\n=== In-processing MLP: EG (Equalized Odds) ===")
print(m_mlp_eo["by_group"])
print(f"Accuracy: {m_mlp_eo['acc']:.4f} | DP diff: {m_mlp_eo['dp']:.4f} | EO diff: {m_mlp_eo['eo']:.4f}")

# 2) EG with Demographic Parity
eg_dp_mlp = ExponentiatedGradient(
    estimator=clone(mlp),
    constraints=DemographicParity(),
    eps=0.01,
    max_iter=50
)
eg_dp_mlp.fit(X_train_ready, y_train, sensitive_features=A_train)

try:
    y_pred_mlp_dp = eg_dp_mlp.predict(X_test_ready, random_state=42)
except TypeError:
    y_pred_mlp_dp = eg_dp_mlp.predict(X_test_ready)

m_mlp_dp = eval_fairness(y_test, y_pred_mlp_dp, A_test)

print("\n=== In-processing MLP: EG (Demographic Parity) ===")
print(m_mlp_dp["by_group"])
print(f"Accuracy: {m_mlp_dp['acc']:.4f} | DP diff: {m_mlp_dp['dp']:.4f} | EO diff: {m_mlp_dp['eo']:.4f}")

# 3) Summary Table
summary_mlp = pd.DataFrame([
    {"model":"MLP Baseline",  "accuracy":m_mlp_base["acc"], "dp_diff":m_mlp_base["dp"], "eo_diff":m_mlp_base["eo"]},
    {"model":"MLP + EG (EO)", "accuracy":m_mlp_eo["acc"],   "dp_diff":m_mlp_eo["dp"],   "eo_diff":m_mlp_eo["eo"]},
    {"model":"MLP + EG (DP)", "accuracy":m_mlp_dp["acc"],   "dp_diff":m_mlp_dp["dp"],   "eo_diff":m_mlp_dp["eo"]},
]).round(4)

print("\n=== MLP: Baseline vs In-processing (EG) ===")
print(summary_mlp)

=== Baseline (MLP) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.666667  0.03125  0.666667       0.131579  0.921053
1    0.802083  0.22000  0.802083       0.602740  0.794521
Accuracy: 0.8207 | DP diff: 0.4712 | EO diff: 0.1888

=== In-processing MLP: EG (Equalized Odds) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.666667  0.03125  0.666667       0.131579  0.921053
1    0.802083  0.22000  0.802083       0.602740  0.794521
Accuracy: 0.8207 | DP diff: 0.4712 | EO diff: 0.1888

=== In-processing MLP: EG (Demographic Parity) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.666667  0.03125  0.666667       0.131579  0.921053
1    0.812500  0.20000  0.812500       0.602740  0.808219
Accuracy: 0.8315 | DP diff: 0.4712 | EO diff: 0.1688

=== MLP: Baseline vs 

### MLP In-Processing Bias Mitigation Results  

### Summary

| Model             | Accuracy | DP diff | EO diff | Interpretation                                                     |
|-------------------|:--------:|:-------:|:-------:|--------------------------------------------------------------------|
| **MLP Baseline**  | 0.8207   | 0.4712  | 0.1888  | Good accuracy; **large DP gap** and **moderate EO**.               |
| **MLP + EG (EO)** | 0.8207   | 0.4712  | 0.1888  | **Identical to baseline** → EO constraint had **no effect**.       |
| **MLP + EG (DP)** | 0.8315   | 0.4712  | 0.1688  | **Accuracy +1.1 pts**; **EO improves slightly**; **DP unchanged**. |

### Summary:
- **Selection disparity persists:** Female sel. **0.132** vs Male **0.603** → **DP = 0.471** (~**4.6×** higher for males) across all runs.
- **EG (EO)** did not move metrics — constraints likely **not binding**.
- **EG (DP)** nudged **EO down** (0.1888 → **0.1688**) and **accuracy up** (0.8207 → **0.8315**), but **did not reduce DP**.

**Takeaway:** With current settings, MLP **resists DP reduction** via in-processing. 

---

### Bias mitigation MLP: Inprocessing: Grid Search

In [27]:
from sklearn.base import clone
from fairlearn.reductions import GridSearch, EqualizedOdds, DemographicParity

# 1) GridSearch with Equalized Odds (MLP)
gs_eo_mlp = GridSearch(
    estimator=clone(adammlp),                 # unfitted clone of your MLP (inherits random_state=42)
    constraints=EqualizedOdds(),
    selection_rule="tradeoff_optimization",  
    constraint_weight=0.5,                   # trade-off weight (0..1); tune as needed
    grid_size=15                             
)
gs_eo_mlp.fit(X_train_ready, y_train, sensitive_features=A_train)
try:
    y_pred_gs_eo_mlp = gs_eo_mlp.predict(X_test_ready, random_state=42)
except TypeError:
    y_pred_gs_eo_mlp = gs_eo_mlp.predict(X_test_ready)

m_gs_eo_mlp = eval_fairness(y_test, y_pred_gs_eo_mlp, A_test)
print("\n=== In-processing MLP: GridSearch (Equalized Odds) ===")
print(m_gs_eo_mlp["by_group"])
print(f"Accuracy: {m_gs_eo_mlp['acc']:.4f} | DP diff: {m_gs_eo_mlp['dp']:.4f} | EO diff: {m_gs_eo_mlp['eo']:.4f}")

# 2) GridSearch with Demographic Parity (MLP)
gs_dp_mlp = GridSearch(
    estimator=clone(adammlp),
    constraints=DemographicParity(),
    selection_rule="tradeoff_optimization",
    constraint_weight=0.5,
    grid_size=15
)
gs_dp_mlp.fit(X_train_ready, y_train, sensitive_features=A_train)
try:
    y_pred_gs_dp_mlp = gs_dp_mlp.predict(X_test_ready, random_state=42)
except TypeError:
    y_pred_gs_dp_mlp = gs_dp_mlp.predict(X_test_ready)

m_gs_dp_mlp = eval_fairness(y_test, y_pred_gs_dp_mlp, A_test)
print("\n=== In-processing MLP: GridSearch (Demographic Parity) ===")
print(m_gs_dp_mlp["by_group"])
print(f"Accuracy: {m_gs_dp_mlp['acc']:.4f} | DP diff: {m_gs_dp_mlp['dp']:.4f} | EO diff: {m_gs_dp_mlp['eo']:.4f}")

# 3) Compare with existing MLP runs (baseline + EG)
summary_mlp = pd.concat([
    summary_mlp,
    pd.DataFrame([
        {"model":"MLP + GS (EO)", "accuracy":m_gs_eo_mlp["acc"], "dp_diff":m_gs_eo_mlp["dp"], "eo_diff":m_gs_eo_mlp["eo"]},
        {"model":"MLP + GS (DP)", "accuracy":m_gs_dp_mlp["acc"], "dp_diff":m_gs_dp_mlp["dp"], "eo_diff":m_gs_dp_mlp["eo"]},
    ]).round(4)
], ignore_index=True)

print("\n=== MLP: Baseline vs EG vs GS ===")
print(summary_mlp)


=== In-processing MLP: GridSearch (Equalized Odds) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.666667  0.03125  0.666667       0.131579  0.921053
1    0.802083  0.22000  0.802083       0.602740  0.794521
Accuracy: 0.8207 | DP diff: 0.4712 | EO diff: 0.1888

=== In-processing MLP: GridSearch (Demographic Parity) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.666667  0.03125  0.666667       0.131579  0.921053
1    0.802083  0.22000  0.802083       0.602740  0.794521
Accuracy: 0.8207 | DP diff: 0.4712 | EO diff: 0.1888

=== MLP: Baseline vs EG vs GS ===
           model  accuracy  dp_diff  eo_diff
0   MLP Baseline    0.8207   0.4712   0.1888
1  MLP + EG (EO)    0.8207   0.4712   0.1888
2  MLP + EG (DP)    0.8315   0.4712   0.1688
3  MLP + GS (EO)    0.8207   0.4712   0.1888
4  MLP + GS (DP)    0.8207   0.4712   0.1888


### MLP — In-Processing vs GridSearch 

#### Comparative table (vs. Baseline)
| Model          | Accuracy | ΔAcc (pp) | DP diff | ΔDP | EO diff | ΔEO  | Notes                           |
|----------------|:--------:|:---------:|:-------:|:---:|:-------:|:----:|----------------------------------|
| Baseline (MLP) | 0.8207   |   –       | 0.4712  |  –  | 0.1888  |  –   | Reference                        |
| EG (EO)        | 0.8207   |  +0.00    | 0.4712  | 0.00| 0.1888  | 0.00 | **No change** vs baseline        |
| EG (DP)        | 0.8315   | **+1.08** | 0.4712  | 0.00| 0.1688  | **−0.02** | Better EO & accuracy; DP unchanged |
| GS (EO)        | 0.8207   |  +0.00    | 0.4712  | 0.00| 0.1888  | 0.00 | **No change** vs baseline        |
| GS (DP)        | 0.8207   |  +0.00    | 0.4712  | 0.00| 0.1888  | 0.00 | **No change** vs baseline        |

#### Interpretation
- **Only EG (DP)** moved the needle: **EO improved** (0.1888 → **0.1688**) and **accuracy increased** (0.8207 → **0.8315**); **DP stayed high** at 0.4712.
- **EG (EO) and both GS variants** produced **identical predictions** to baseline → constraints likely **not binding** or reweighting not affecting the MLP.

---

### Bias mitigation MLP: Postprocessing: Threshold Optimizer

In [28]:
from fairlearn.postprocessing import ThresholdOptimizer
import pandas as pd

# 0) Baseline MLP
adammlp.fit(X_train_ready, y_train)
y_mlp_base = adammlp.predict(X_test_ready)
m_mlp_base = eval_fairness(y_test, y_mlp_base, A_test)

print("=== Baseline (MLP) ===")
print(m_mlp_base["by_group"])
print(f"Accuracy: {m_mlp_base['acc']:.4f} | DP diff: {m_mlp_base['dp']:.4f} | EO diff: {m_mlp_base['eo']:.4f}")

# 1) Post-processing: Equalized Odds
post_mlp_eo = ThresholdOptimizer(
    estimator=adammlp,
    constraints="equalized_odds",
    predict_method="predict_proba",
    grid_size=200,
    flip=True
)
post_mlp_eo.fit(X_train_ready, y_train, sensitive_features=A_train)
y_mlp_eo = post_mlp_eo.predict(X_test_ready, sensitive_features=A_test)
m_mlp_eo = eval_fairness(y_test, y_mlp_eo, A_test)

print("\n=== MLP + Post-processing (Equalized Odds) ===")
print(m_mlp_eo["by_group"])
print(f"Accuracy: {m_mlp_eo['acc']:.4f} | DP diff: {m_mlp_eo['dp']:.4f} | EO diff: {m_mlp_eo['eo']:.4f}")

# 2) Post-processing: Demographic Parity
post_mlp_dp = ThresholdOptimizer(
    estimator=adammlp,
    constraints="demographic_parity",
    predict_method="predict_proba",
    grid_size=200,
    flip=True
)
post_mlp_dp.fit(X_train_ready, y_train, sensitive_features=A_train)
y_mlp_dp = post_mlp_dp.predict(X_test_ready, sensitive_features=A_test)
m_mlp_dp = eval_fairness(y_test, y_mlp_dp, A_test)

print("\n=== MLP + Post-processing (Demographic Parity) ===")
print(m_mlp_dp["by_group"])
print(f"Accuracy: {m_mlp_dp['acc']:.4f} | DP diff: {m_mlp_dp['dp']:.4f} | EO diff: {m_mlp_dp['eo']:.4f}")

# 3) Summary Table
summary_mlp_post = pd.DataFrame([
    {"model":"MLP Baseline",       "accuracy":m_mlp_base["acc"], "dp_diff":m_mlp_base["dp"], "eo_diff":m_mlp_base["eo"]},
    {"model":"MLP + Post (EO)",    "accuracy":m_mlp_eo["acc"],   "dp_diff":m_mlp_eo["dp"],   "eo_diff":m_mlp_eo["eo"]},
    {"model":"MLP + Post (DP)",    "accuracy":m_mlp_dp["acc"],   "dp_diff":m_mlp_dp["dp"],   "eo_diff":m_mlp_dp["eo"]},
]).round(4)

print("\n=== MLP: Baseline vs Post-processing ===")
print(summary_mlp_post)

=== Baseline (MLP) ===
          TPR      FPR    Recall  SelectionRate  Accuracy
Sex                                                      
0    0.666667  0.03125  0.666667       0.131579  0.921053
1    0.802083  0.22000  0.802083       0.602740  0.794521
Accuracy: 0.8207 | DP diff: 0.4712 | EO diff: 0.1888

=== MLP + Post-processing (Equalized Odds) ===
         TPR      FPR   Recall  SelectionRate  Accuracy
Sex                                                    
0    0.50000  0.09375  0.50000       0.157895  0.842105
1    0.84375  0.22000  0.84375       0.630137  0.821918
Accuracy: 0.8261 | DP diff: 0.4722 | EO diff: 0.3438

=== MLP + Post-processing (Demographic Parity) ===
         TPR   FPR   Recall  SelectionRate  Accuracy
Sex                                                 
0    0.50000  0.00  0.50000       0.078947  0.921053
1    0.84375  0.22  0.84375       0.630137  0.821918
Accuracy: 0.8424 | DP diff: 0.5512 | EO diff: 0.3438

=== MLP: Baseline vs Post-processing ===
        

### MLP — Post-Processing: Threshold Optimizer  


### Summary

| Model               | Accuracy | DP diff | EO diff | Notes                                                                 |
|---------------------|:--------:|:-------:|:-------:|------------------------------------------------------------------------|
| **Baseline**        | 0.8207   | 0.4712  | 0.1888  | Notable outcome disparity (DP) and moderate error-rate gap (EO).       |
| **Post (EO)**       | 0.8261   | 0.4722  | 0.3438  | Acc **+0.54 pp**; **EO worsens** (+0.155); DP ~unchanged (+0.001).     |
| **Post (DP)**       | 0.8424   | 0.5512  | 0.3438  | Acc **+2.17 pp**; **DP worsens** (+0.080); **EO worsens** (+0.155).    |

### Interpretation
- **Equalized Odds post-processing** slightly increases accuracy but **substantially increases EO** and leaves **DP unchanged/slightly worse**.  
- **Demographic Parity post-processing** raises accuracy more, yet **worsens both DP and EO** (female selection drops to **0.079** while male stays high **0.630** → **DP ↑ to 0.551**).  
- Both post-processing variants converge to **EO = 0.3438**, driven by a large **TPR gap** (Female **0.50** vs Male **0.844**).

**Takeaway:** With current scores/settings, post-processing **degrades fairness** (EO—and for DP constraint also DP) despite small accuracy gains. 

---

### Overall Comparison:

### Overall Bias-Mitigation Comparison (AIF360) — Gender Bias in CVD Prediction

**Metric keys:**  
- **DP diff** (Demographic Parity difference): outcome-rate gap across genders (lower = fairer).  
- **EO diff** (Equalized Odds difference): error-rate gap across genders (lower = fairer).  
- **Accuracy**: utility measure (higher = better).  

---

### Aggregated Summary Across Models

| Model / Technique         | Accuracy | DP diff | EO diff | Verdict |
|---------------------------|:--------:|:-------:|:-------:|---------|
| **PCA+KNN Baseline**      | 0.8315   | 0.4301  | 0.1146  | Reference; DP large, EO moderate |
| KNN + Post (DP/EO)        | 0.8315   | 0.4301  | 0.1146  | ❌ No effect — 0% label flips |
| KNN + CorrelationRemover  | 0.8043   | 0.4117  | 0.3021  | ❌ Acc ↓, EO worsens |
| **Decision Tree Baseline**| 0.8261   | 0.3439  | 0.0938  | Reference; moderate DP, small EO |
| DT + EG (EO)              | 0.8261   | 0.3497  | 0.0729  | ✅ Best DT if acc must hold — EO improves modestly |
| DT + EG (DP)              | 0.8098   | 0.2823  | 0.0413  | ✅ Strongest DT: both DP & EO ↓; small acc cost |
| DT + Post (EO/DP)         | 0.7174 / 0.8207 | 0.3079 / 0.3659 | 0.2917 / 0.1562 | ❌ Either acc ↓ or fairness worsens |
| DT + GS (EO/DP)           | 0.7337 / 0.8043 | 0.2390 / 0.4239 | 0.0521 / 0.1813 | ❌ EO run has lowest DP/EO but acc collapse; DP run worsens fairness |
| **Random Forest Baseline**| 0.8424   | 0.4106  | 0.1088  | Strong acc; large DP; moderate EO |
| RF + EG (EO/DP)           | 0.8424   | 0.4106  | 0.1088  | ❌ No effect |
| RF + GS (EO i=29)         | 0.8424   | 0.3911  | 0.0775  | ✅ Best EO fairness at baseline acc |
| RF + GS (EO i=40)         | 0.8478   | 0.4037  | 0.0888  | ✅ Best accuracy with EO ↓; DP still moderate |
| RF + GS (DP i=31)         | 0.8315   | 0.3911  | 0.0975  | ✅ Balanced DP + EO ↓; small acc cost |
| RF + Post (EO/DP)         | 0.8424   | 0.4632  | 0.1400  | ❌ Fairness worsens, acc unchanged |
| **MLP Baseline**          | 0.8207   | 0.4712  | 0.1888  | Reference; weakest baseline fairness |
| MLP + EG (EO/DP)          | 0.8207 / 0.8315 | 0.4712 | 0.1888 / 0.1688 | ❌ Minimal/no gains |
| MLP + GS (EO/DP)          | 0.8207   | 0.4712  | 0.1888  | ❌ No effect |
| MLP + Post (EO/DP)        | 0.8261 / 0.8424 | 0.4722 / 0.5512 | 0.3438 / 0.3438 | ❌ EO diff worsened sharply |

---

### What Worked Best

- **Decision Tree + EG (DP):** Both DP and EO improved (**0.344 → 0.282, 0.094 → 0.041**) with only −1.6 pp acc.  
- **Decision Tree + EG (EO):** EO ↓ to 0.073 at baseline accuracy — safe improvement.  
- **Random Forest + GS (EO i=29):** EO ↓ to 0.078 with DP ↓, accuracy stable — balanced choice.  
- **Random Forest + GS (EO i=40):** Highest acc (0.848) with EO ↓ to 0.089.  
- **Random Forest + GS (DP i=31):** DP ↓ and EO ↓ jointly, small acc drop.

---

### What Did Not Help

- **PCA+KNN post-processing:** 0% label flips → no effect.  
- **CorrelationRemover (KNN):** Hurt accuracy and worsened EO.  
- **RF + EG (EO/DP):** Constraints not binding → baseline repeated.  
- **RF + Post (EO/DP):** Fairness worsened, accuracy unchanged.  
- **MLP EG/GS/Post:** Either no movement or worsened EO/DP.  

---

### Practical Implications for CVD Prediction

- **If priority = EO parity (error-rate fairness):**  
  - Use **DT + EG (DP)** (strongest EO ↓ with small acc cost), or  
  - **RF + GS (EO i=29/40)** (stable/high accuracy with EO ↓).  

- **If priority = DP parity (outcome-rate fairness):**  
  - **DT + EG (DP)** (lowest DP), or  
  - **RF + GS (DP i=31)** (balanced DP ↓ & EO ↓, small acc trade-off).  

- **Avoid:** PCA+KNN fixes, RF Post/EG, and MLP interventions — none provided meaningful fairness gains.  

---

### Final Recommendation

For fair and accurate gender-sensitive CVD prediction:  
- **Primary choice:** **Random Forest + GS (EO i=29 or i=40)** for balanced EO ↓ with high accuracy.  
- **Secondary:** **Decision Tree + EG (DP)** if interpretability and consistent DP+EO improvements matter.  
- **Tertiary:** **RF + GS (DP i=31)** if DP parity is the key objective.  
- **KNN & MLP:** No reliable fairness improvements — better left out in clinical deployment.  

---